In [1]:
import os

def determine_fragments(pdb):
    DNA_A = ''
    DNA_B = ''
    protein_A = ''
    protein_B = ''
    c = 1
    NAs = 'DA DT DC DG DA5 DT5 DC5 DG5 DA3 DT3 DC3 DG3'.split(' ')
    AAs = 'ALA ARG ASP ASN CYS GLU GLY HIS HIE HID ISO LEU LYS MET PHE PRO SER THR TRY TYR VAL SEL'.split(' ')
    one_NAs = {'DA':'A', 'DT':'T', 'DC':'C', 'DG':'G',
               'DA5':'A', 'DT5':'T', 'DC5':'C', 'DG5':'G',
               'DA3':'A', 'DT3':'T', 'DC3':'C', 'DG3':'G',
               'A':'A', 'U':'U', 'C':'C', 'G':'G',
               'A5':'A', 'U5':'U', 'C5':'C', 'G5':'G',
               'A3':'A', 'U3':'U', 'C3':'C', 'G3':'G'}

    one_AAs = {'ALA':'A', 'ARG':'R', 'ASP':'D', 'ASN':'N',
               'CYS':'C', 'GLU':'E', 'GLN':'Q', 'GLY':'G',
               'HIS':'H', 'HIE':'H', 'HID':'H', 'ILE':'I',
               'LEU':'L', 'LYS':'K', 'MET':'M', 'PHE':'F',
               'PRO':'P', 'SER':'S', 'THR':'T', 'TRP':'W',
               'TYR':'Y', 'VAL':'V', 'SEL':'Z'}
    pdb = [i for i in pdb if (" O5'" in i or 'TER' in i or ' CA' in i)]           
    for line in range(len(pdb)):
        split = pdb[line].split()
        splot = pdb[line-1].split() 
        if 'TER' in pdb[line]:
            c+=1
        if split[0] == 'ATOM':
            if (split[3] in NAs or split[3] in AAs):
                if c == 1:
                    DNA_A+=one_NAs[split[3]]
                if c == 2:
                    DNA_B+=one_NAs[split[3]]
                if c == 3:
                    protein_A+=one_AAs[split[3]]
                if c == 4:
                    protein_B+=one_AAs[split[3]]
    # if c == 4:
    #     return DNA_A, DNA_B, protein_A
    # if c == 5:
    #     return DNA_A, DNA_B, protein_A, protein_B
    return DNA_A, DNA_B, protein_A, protein_B

temp = open('/Users/HQ/orange/Reza/TF/NAR/1azp/14-two-sided-dummy-contacts/TEMPLATES/1azp-BDNA.pdb').readlines()
a, b, c, d = determine_fragments(temp)

In [3]:
systems = ['1a74', '1azp', '1by4', '1bgb',  '2b0d', '1cdw', '1dh3', '1jj4', '1r4o', '1r4r', '1zme', '2dgc', '2r1j', '3cro', '1ysa', '2dgc']
# systems = ['1a74', '1azp', '1by4', '1bgb',  '2b0d', '1dh3', '1jj4', '1r4o', '1r4r', '1zme', '2r1j', '3cro', '1ysa', '2dgc']
for system in systems:
    dimer = ''
    template = open('/Users/HQ/orange/Reza/TF/NAR/%s/14-two-sided-dummy-contacts/TEMPLATES/%s-BDNA.pdb'%(system, system)).readlines()
    DNA_A, DNA_B, protein_A, protein_B = determine_fragments(template)
    with open('/Users/HQ/orange/Reza/TF/RF2NA/%s/DNA_A.fa'%(system), 'w') as outfile:
        outfile.write('> DNA_A\n')
        outfile.write(DNA_A+'\n')
    outfile.close()
    with open('/Users/HQ/orange/Reza/TF/RF2NA/%s/DNA_B.fa'%(system), 'w') as outfile:
        outfile.write('> DNA_B\n')
        outfile.write(DNA_A+'\n')
    outfile.close()
    with open('/Users/HQ/orange/Reza/TF/RF2NA/%s/protein_A.fa'%(system), 'w') as outfile:
        outfile.write('> protein_A\n')
        outfile.write(protein_A+'\n')
    outfile.close()
    with open('/Users/HQ/orange/Reza/TF/RF2NA/%s/protein_AB.fa'%(system), 'w') as outfile:
        outfile.write('> protein_AB\n')
        outfile.write(protein_A+protein_B+'\n')
    outfile.close()
    if protein_B != '':
        dimer = 'protein_B.fa'
        with open('/Users/HQ/orange/Reza/TF/RF2NA/%s/protein_B.fa'%(system), 'w') as outfile:
            outfile.write('> protein_B\n')
            outfile.write(protein_B+'\n')
    outfile.close()
    with open('/Users/HQ/orange/Reza/TF/RF2NA/%s/run-RF2NA.sh'%(system), 'w') as outfile:
        outfile.write('''#!/bin/bash
#SBATCH --job-name=%s-RF2NA                 # Job name
#SBATCH --mail-type=FAIL          # Mail events (NONE, BEGIN, END, FAIL, ALL)
#SBATCH --mail-user=reza.esmaeeli@ufl.edu   # Where to send mail
#SBATCH --ntasks=1                          # Number of MPI ranks
#SBATCH --cpus-per-task=1                   # Number of cores per MPI rank 
#SBATCH --nodes=1                           # Number of nodes
#SBATCH --ntasks-per-node=1                 # How many tasks on each node
#SBATCH --ntasks-per-socket=1               # How many tasks on each CPU or socket
#SBATCH --distribution=cyclic:cyclic        # Distribute tasks cyclically on nodes and sockets
#SBATCH --mem-per-cpu=32000mb               # Memory per processor
#SBATCH --time=7-00:00:00                   # Time limit hrs:min:sec
#SBATCH --output=RF2NA.log                  # Standard output and error log
#SBATCH --qos=alberto.perezant

pwd; hostname; date

deactivate
conda deactivate
module purge
module load conda
conda activate RF2NA

rm -rf output

/orange/alberto.perezant/Reza/RF2NA/software/RoseTTAFold2NA-main/run_RF2NA.sh output D:DNA_A.fa D:DNA_B.fa protein_A.fa %s


'''%(system.upper(), dimer))
    outfile.close()

    with open('/Users/HQ/orange/Reza/TF/RF2NA/%s/run-RF2NA-combined.sh'%(system), 'w') as outfile:
        outfile.write('''#!/bin/bash
#SBATCH --job-name=%s-RF2NA-combined        # Job name
#SBATCH --mail-type=FAIL          # Mail events (NONE, BEGIN, END, FAIL, ALL)
#SBATCH --mail-user=reza.esmaeeli@ufl.edu   # Where to send mail
#SBATCH --ntasks=1                          # Number of MPI ranks
#SBATCH --cpus-per-task=1                   # Number of cores per MPI rank 
#SBATCH --nodes=1                           # Number of nodes
#SBATCH --ntasks-per-node=1                 # How many tasks on each node
#SBATCH --ntasks-per-socket=1               # How many tasks on each CPU or socket
#SBATCH --distribution=cyclic:cyclic        # Distribute tasks cyclically on nodes and sockets
#SBATCH --mem-per-cpu=32000mb               # Memory per processor
#SBATCH --time=7-00:00:00                   # Time limit hrs:min:sec
#SBATCH --output=RF2NA-combined.log                  # Standard output and error log
#SBATCH --qos=alberto.perezant

pwd; hostname; date

deactivate
conda deactivate
module purge
module load conda
conda activate RF2NA

rm -rf output-combined

/orange/alberto.perezant/Reza/RF2NA/software/RoseTTAFold2NA-main/run_RF2NA.sh output-combined D:DNA_A.fa D:DNA_B.fa protein_AB.fa


'''%(system.upper()))
    outfile.close()